In [8]:
from sqlalchemy import create_engine
import pandas as pd

In [5]:
def connect_to_database():
    """Function for connecting to the SQL database"""

    # Create variables to store cretentials
    USER = "admin"
    DB = "dbikes"
    PORT = "3306"
    URL = "database-1.ctesjcult8dm.eu-west-1.rds.amazonaws.com"

    # Read in password from text file
    with open('../mysql_password.txt') as f:
        PASSWORD = ''.join(f.readlines())
        PASSWORD = str(PASSWORD).split()[0]

    # Create engine using credentials
    engine = create_engine(
        "mysql+mysqlconnector://{}:{}@{}:{}/{}".format(USER, PASSWORD, URL, PORT, DB), echo=True)

    # Create connection using engine and return connection
    conn = engine.connect()
    return conn


In [7]:
engine = connect_to_database()

2022-03-23 11:42:52,903 INFO sqlalchemy.engine.Engine SHOW VARIABLES LIKE 'sql_mode'
2022-03-23 11:42:52,905 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-03-23 11:42:52,921 INFO sqlalchemy.engine.Engine SHOW VARIABLES LIKE 'lower_case_table_names'
2022-03-23 11:42:52,922 INFO sqlalchemy.engine.Engine [generated in 0.00594s] {}
2022-03-23 11:42:52,948 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2022-03-23 11:42:52,950 INFO sqlalchemy.engine.Engine [raw sql] {}


In [10]:
df = pd.read_sql_query("SELECT DISTINCT number FROM dbikes.station", engine)
df.head()

2022-03-23 11:45:38,579 INFO sqlalchemy.engine.Engine SELECT DISTINCT number FROM dbikes.station
2022-03-23 11:45:38,581 INFO sqlalchemy.engine.Engine [raw sql] {}


,number
0,2
1,3
2,4
3,5
4,6
